In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from csv import writer

In [2]:
station_latitude = pd.read_csv("station_data.csv",index_col=0)
station_latitude = station_latitude[['device_id','latitude','longitude']]
station_latitude_dic = station_latitude.set_index('device_id').T.to_dict('list')
station_latitude_dic = {tuple(v):k for k,v in station_latitude_dic.items()}

all_latitudes = list(station_latitude_dic.keys())
def find_k_nearest(src, latitudes, k):
    def euclidean_dist(p1,p2):
        return (p1[0]-p2[0])**2+(p1[1]-p2[1])**2
    all_distances = [ (coord[0],coord[1], euclidean_dist(src,coord)) for coord in latitudes]
    return sorted(all_distances, key= lambda x:x[2])[1:k+1] #skips 1st point which is itself


k_nearest_dic = {}

for coord, stn in station_latitude_dic.items():
    k_nearest_dic[stn] = list(map(lambda x: (station_latitude_dic[x[:2]],x[2]), find_k_nearest(coord, all_latitudes, 5) ))
k_nearest_dic


{'S77': [('S223', 0.00013491920000006572),
  ('S222', 0.00013876849999994398),
  ('S120', 0.0002564515999999745),
  ('S226', 0.0004343725000000764),
  ('S79', 0.0006549799999997879)],
 'S109': [('S216', 0.00027998660000008377),
  ('S08', 0.0005280999999997645),
  ('S217', 0.0007122037000000591),
  ('S219', 0.0007543610000001618),
  ('S88', 0.0011366899999999854)],
 'S64': [('S218', 0.000399022600000149),
  ('S114', 0.000923849999999812),
  ('S40', 0.0013436623999999104),
  ('S100', 0.0013491025000001455),
  ('S121', 0.001524009999999613)],
 'S90': [('S120', 0.00010853960000001625),
  ('S213', 0.00011508889999999012),
  ('S111', 0.0003758624999998223),
  ('S46', 0.0005785000000002053),
  ('S223', 0.0006418792000002979)],
 'S61': [('S212', 0.00021874409999983975),
  ('S113', 0.00040060040000012306),
  ('S84', 0.000943780000000015),
  ('S215', 0.001093466599999638),
  ('S78', 0.0012179018000000772)],
 'S114': [('S121', 0.000107847999999967),
  ('S218', 0.0006541305999995478),
  ('S229', 0

In [3]:
rain_data = pd.read_csv("rain_data_full.csv", index_col=0)
rain_data

,date,time,station,value
0,2023-02-03,00:05:00,S77,0.0
1,2023-02-03,00:10:00,S77,0.0
2,2023-02-03,00:15:00,S77,0.0
3,2023-02-03,00:20:00,S77,0.0
4,2023-02-03,00:25:00,S77,0.0
...,...,...,...,...
25727377,2020-09-09,23:35:00,S105,NaN
25727378,2020-09-09,23:40:00,S105,NaN
25727379,2020-09-09,23:45:00,S105,NaN
25727380,2020-09-09,23:50:00,S105,NaN


In [4]:
rain_data_pivot = rain_data.pivot(index=["date","time"], columns="station", values="value")
rain_data_pivot

station              S07  S08  S100  S102  S104  S105  S106  S107  S108  S109  \
date       time                                                                 
2020-01-01 00:05:00  0.0  0.0   0.0   NaN   0.0   0.0   0.0   0.0   0.0   0.0   
           00:10:00  0.0  0.0   0.0   NaN   0.0   0.0   0.0   0.0   0.0   0.0   
           00:15:00  0.0  0.0   0.0   NaN   0.0   0.0   0.0   NaN   0.0   0.0   
           00:20:00  0.0  0.0   0.0   NaN   0.0   0.0   0.0   0.0   0.0   0.0   
           00:25:00  0.0  0.0   0.0   NaN   0.0   0.0   0.0   0.0   0.0   0.0   
...                  ...  ...   ...   ...   ...   ...   ...   ...   ...   ...   
2023-02-28 23:35:00  NaN  2.4   0.0   NaN   0.2   NaN   6.2   0.8   1.0   3.4   
           23:40:00  NaN  4.2   0.0   NaN   0.2   NaN   5.4   1.8   1.6   3.8   
           23:45:00  NaN  3.4   0.0   NaN   0.0   NaN   4.8   2.8   3.0   1.6   
           23:50:00  NaN  1.6   0.2   NaN   0.0   NaN   2.4   2.0   3.4   1.8   
           23:55:00  NaN  2.2   0.0   NaN   0.4   NaN   4.0   1.0   3.6   1.8   

station              ...  S81  S82  S84  S88  S89  S90  S900  S91  S92  S94  
date       time      ...                                                     
2020-01-01 00:05:00  ...  0.0  0.0  0.0  0.0  0.0  0.0   0.0  0.0  0.0  0.0  
           00:10:00  ...  0.0  0.0  0.0  0.0  0.0  0.0   0.0  0.0  0.0  0.0  
           00:15:00  ...  0.0  0.0  0.0  0.0  0.0  0.0   0.0  0.0  0.0  0.0  
           00:20:00  ...  0.0  0.0  0.0  0.0  0.0  0.0   0.0  0.0  0.0  0.0  
           00:25:00  ...  0.0  0.0  0.0  0.0  0.0  0.0   0.0  0.0  0.0  0.0  
...                  ...  ...  ...  ...  ...  ...  ...   ...  ...  ...  ...  
2023-02-28 23:35:00  ...  0.6  NaN  0.8  3.2  1.0  1.6   2.0  NaN  NaN  0.2  
           23:40:00  ...  4.2  NaN  0.6  4.0  0.4  3.8   2.8  NaN  NaN  0.2  
           23:45:00  ...  4.0  NaN  0.6  1.6  0.2  4.6   NaN  NaN  NaN  0.8  
           23:50:00  ...  3.2  NaN  0.6  2.0  0.6  5.4   0.4  NaN  NaN  2.4  
           23:55:00  ...  2.4  NaN  2.6  2.2  1.4  2.6   0.6  NaN  NaN  4.6  

[317622 rows x 81 columns]

In [5]:
rain_data_pivot = rain_data_pivot.to_dict('index')


In [6]:
def get_time_tuples(date_time_tuple):
    date_str, time_str = date_time_tuple
    datetime_obj = datetime.strptime(date_str + ' ' + time_str, '%Y-%m-%d %H:%M:%S')
    result = []
    for i in range(1,7):
        new_datetime_obj = datetime_obj - timedelta(minutes=i*5)
        new_date_str = new_datetime_obj.strftime('%Y-%m-%d')
        new_time_str = new_datetime_obj.strftime('%H:%M:%S')
        result.append((new_date_str, new_time_str))
    return result


In [7]:
header = ["date", "time", "station", "value"]
for time_indx in range(1,7):
    for stn_indx in range(1,6):
        header.append(f'T{time_indx}S{stn_indx}_time')
        header.append(f'T{time_indx}S{stn_indx}_station number')
        header.append(f'T{time_indx}S{stn_indx}_dist')
        header.append(f'T{time_indx}S{stn_indx}_value')

with open('sliding_window_data.csv',"a", encoding='utf-8', newline='') as file:
    writer_object = writer(file)
    writer_object.writerow(header)
    file.close()

In [8]:
lst_of_rows = []
for i in range(1,len(rain_data)):
    orig_date, orig_time, orig_stn, orig_val = rain_data.iloc[i,0], rain_data.iloc[i,1], rain_data.iloc[i,2], rain_data.iloc[i,3]
    nearest_k_stns = k_nearest_dic[orig_stn]
    newRow = [orig_date, orig_time, orig_stn, orig_val]
    # rowIndex = rain_data.index[i]
    for time_indx,(date,time_stamp) in enumerate(get_time_tuples((orig_date, orig_time)),start=1):
        for (stn,dist) in nearest_k_stns:
            newRow.append(-5*time_indx)
            newRow.append(stn)
            newRow.append(dist)
            newRow.append(rain_data_pivot.get((date,time_stamp),{}).get(stn,np.nan))
    lst_of_rows.append(newRow)
    if i % 1000 == 0:
        print(f"Currently writing up till iteration {i}")
        #write multiple rows
        with open('sliding_window_data.csv',"a", encoding='utf-8', newline='') as file:
            writer_object = writer(file)
            writer_object.writerows(lst_of_rows)
        lst_of_rows = []

#write last batch
with open('sliding_window_data.csv',"a", encoding='utf-8', newline='') as file:
    writer_object = writer(file)
    writer_object.writerows(lst_of_rows)

Currently writing up till iteration 1000
Currently writing up till iteration 2000
Currently writing up till iteration 3000
Currently writing up till iteration 4000
Currently writing up till iteration 5000
Currently writing up till iteration 6000
Currently writing up till iteration 7000
Currently writing up till iteration 8000
Currently writing up till iteration 9000
Currently writing up till iteration 10000
Currently writing up till iteration 11000
Currently writing up till iteration 12000
Currently writing up till iteration 13000
Currently writing up till iteration 14000
Currently writing up till iteration 15000
Currently writing up till iteration 16000
Currently writing up till iteration 17000
Currently writing up till iteration 18000
Currently writing up till iteration 19000
Currently writing up till iteration 20000
Currently writing up till iteration 21000
Currently writing up till iteration 22000
Currently writing up till iteration 23000
Currently writing up till iteration 24000
C

In [ ]:
rain_data.iloc[25727381:,:]

,date,time,station,value
25727381,2020-09-09,23:55:00,S105,NaN


In [ ]:
len(rain_data)

25727382

In [ ]:
get_time_tuples(('2023-02-03', '00:05:00'))

[('2023-02-03', '00:00:00'),
 ('2023-02-02', '23:55:00'),
 ('2023-02-02', '23:50:00'),
 ('2023-02-02', '23:45:00'),
 ('2023-02-02', '23:40:00'),
 ('2023-02-02', '23:35:00')]